In [1]:
from sklearn import datasets

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

In [4]:
from sklearn.tree import DecisionTreeClassifier

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
from sklearn.tree import export_text

In [7]:
from sklearn.model_selection import ShuffleSplit, cross_validate

In [8]:
iris = datasets.load_iris()

In [9]:
dir(iris)

['DESCR',
 'data',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [ ]:
df = pd.DataFrame(
    iris.data,
    columns = iris.feature_names
)

In [ ]:
df['target'] = pd.Series(
    iris.target
)

In [ ]:
df.head(3)

In [ ]:
df['target_values'] = df['target'].apply(
    lambda y:iris.target_names[y]
)

In [ ]:
df.head(3)

In [ ]:
number_of_columns_and_rows = df.shape
number_of_columns_and_rows

In [ ]:
df.sample(n=6,random_state = 42)

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df,test_size = 0.3)

In [ ]:
sample_training_set = df_train.head(5)
sample_training_set

In [ ]:
sample_testing_set = df_test.head(5)
sample_testing_set

In [ ]:
x_train,y_train = df_train[iris.feature_names],df_train['target']
x_test,y_test = df_test[iris.feature_names], df_test['target']

In [ ]:
# it is common to call a classifier intense as clf
clf = DecisionTreeClassifier()

In [ ]:
clf.fit(x_train,y_train)

In [ ]:
# if y_test is the truth then y_test_pred is our estimate of the truth from the model.

y_test_pred = clf.predict(x_test)

In [ ]:
accuracy_score(y_test,y_test_pred)

In [ ]:
pd.DataFrame(
    {
        'feature_names': iris.feature_names,
        'feature_importance': clf.feature_importances_
    }
).sort_values(
    'feature_importance',ascending = False

).set_index('feature_names')

In [ ]:
print(
    export_text(clf, feature_names = iris.feature_names, spacing = 3, decimals = 1)
)

####

In [ ]:
accuracy_scores = []

In [ ]:
for _ in range(100):
    df_train, df_test = train_test_split(df, test_size = 0.3)
    x_train, y_train  = df_train[iris.feature_names], df_train['target']
    x_test, y_test    = df_test[iris.feature_names], df_test['target']
    clf               = DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    accuracy_scores.append(round(accuracy_score(y_test,y_pred),3))
accuracy_scores = pd.Series(accuracy_scores)

In [ ]:
accuracy_scores.plot(
    title = 'Distribution of classifier accuracy',
    kind  = 'box',
)

In [ ]:
print(
    """Average Score: {:.3} [5th percentile: {:.3} & 95th percentile
    :{:.3}]""".format(
    accuracy_scores.mean(),
    accuracy_scores.quantile(.05),
    accuracy_scores.quantile(.05),
    )
)

####

In [ ]:
accuracy_scores = []
rs = ShuffleSplit(n_splits = 100,  test_size=0.3)
for train_index, test_index in rs.split(df):
    x_train, x_test = df.loc[train_index, iris.feature_names], df.loc[test_index, iris.feature_names]
    y_train, y_test = df.loc[train_index, 'target'], df.loc[test_index, 'target']
    clf = DecisionTreeClassifier()
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    accuracy_scores.append(round(accuracy_score(y_test, y_pred), 3))
accuracy_scores = pd.Series(accuracy_scores)

In [ ]:
for max_depth in [1,2,3,4]:
    clf = DecisionTreeClassifier(max_depth=max_depth)
    rs  = ShuffleSplit(n_splits=20, test_size=0.25)
    cv_results = cross_validate(
        clf, x_train, y_train, cv=rs, scoring='accuracy'
    )
    accuracy_scores = pd.Series(cv_results['test_score'])
    print('@ max_depth = {}: accuracy_scores: {}~{}'.format(
        max_depth,
        accuracy_scores.quantile(0.1).round(3),
        accuracy_scores.quantile(0.9).round(3),
    ))

In [ ]:
def plot_decision_boundaries(clf, x, y):
    feature_names = x.columns
    x, y          = x.values, y.values
    x_min, x_max  = x[:,0].min(), x[:,0].max()
    y_min, y_max  = x[:,1].min(), x[:,1].max()    
    step          = 0.02
    xx, yy        = np.meshgrid(
        np.arange(x_min, y_max, step),
        np.arange(y_min, y_max, step)
    )
    z             = clf.predict(np.c_[xx.ravel(),yy.ravel()])
    z             = z.reshape(xx.shape)
    plt.figure(figsize=(12,8))
    plt.contourf(xx,yy,z,cmap='Paired_r',alpha=0.25)
    plt.contour(xx,yy,z,colors='k',linewidths = 0.7)
    plt.scatter(x[:,0],x[:,1],c=y,edgecolors='k')
    plt.title("Tree's Decision Boundaries")
    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[1])

In [ ]:
x = df[['petal width (cm)', 'petal length (cm)']]

In [ ]:
y = df['target']

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(x,y)
plot_decision_boundaries(clf,x,y)